In [1]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
import html2text
import pandas as pd
from nltk import word_tokenize, pos_tag
from MyPackages import functions

##### 1 > train the model on data1 ,validate the model on the other data. 
    o/p table : 
    1c data 2c precisition 3c recall 4c no of 0's and no of 1's 5c f1 score
##### 2 >train the models on 4 randomly choosen data collated together
      use CV to choose appropriate hyperparameters
      Test the model
      1>on the remaining 4 data individually
      2> on a single test data created by appending the 4 test data
##### 3>Test the model on a dataset on gobi and chicken manchurian
       
       
#### The model sequence
     1>DT
     2>RF
     3>Adaboost
     4>SVM
     5>GBM

In [118]:
import os
os.getcwd()

'C:\\Users\\Priyanka G\\CAPSTONE_PROJECT_WORK(2022)'

    1>https://www.yummytummyaarthi.com/paneer-manchurian-recipe-how-to-make/
    2>https://www.vegrecipesofindia.com/paneer-manchurian-dry-recipe/
    3>https://www.indianhealthyrecipes.com/paneer-manchurian-recipe/
    4>https://hebbarskitchen.com/paneer-manchurian-dry-recipe/
    5>https://foodviva.com/chinese-recipes/paneer-manchurian-recipe/
    6>https://www.mydelicious-recipes.com/recipedetail/dry-paneer-manchurian
    7>https://www.tarladalal.com/paneer-in-manchurian-sauce-86r
    8>https://www.awesomecuisine.com/recipes/725/paneer-manchurian/



## Function1 - WorkFlow

#####    1>Takes the url of the website as the argument 
#####     2>Extracts the text format of the raw html file  
#####     3>Data cleanning and preprocessing 
#####     4>Split sentence on the basis of \n encounter 
#####     5>Arrange all the sentence along with the no of \n in each    
#####     6>Create a text file containing the cleanned text
#####     7>Return the list of cleanned string 



In [2]:
def function1(url):
    
    ## Data extraction in the text format
    r2 = requests.get(url).text
    h = html2text.HTML2Text()
    # Ignore converting links from HTML
    h.ignore_links = True
    result = h.handle(r2)
    #result
    result =result.replace(".","")
    
    ### Data cleanning and preprocessing
    r=[]
    from nltk.tokenize import sent_tokenize
    sent_tokenize=sent_tokenize(result)
    for i in range(len(sent_tokenize)):
            sent_tokenize[i] = sent_tokenize[i].lower() 
            sent_tokenize[i] = re.sub(r'[^a-zA-Z0-9\n/]+'," ",sent_tokenize[i])
            #sent_tokenize[i] = re.sub(r"[▢,#,*,%,!,$,),},?,-,@,:,{,;,=,&,+,_,[,x,(,',<,>,~,`,^,|,▼]","",sent_tokenize[i])
            #sent_tokenize[i] = re.sub(r"['»']","",sent_tokenize[i])
            #sent_tokenize[i] = re.sub(r"-","",sent_tokenize[i])
            #sent_tokenize[i] = re.sub(r"]","",sent_tokenize[i])
            #sent_tokenize[i] = re.sub(r"['\']","",sent_tokenize[i])

    ### remove the additional spaces         
    for i in sent_tokenize:
        r.append(i.strip())
    fr = list(filter(lambda item: item.strip(), r))
    #fr
    
    ###splitting the sentence on the basis of \n
    w=[]
    for n in range(len(fr)):
        w.append(fr[n].splitlines(True))

    ##flattening the list
    f = [item for sublist in w for item in sublist]
    #f
    
    ###condition checking for retaining all the \n in the respective sentences
    o=[]
    for i in range(len(f)):
        if i != len(f)-1:
            if (f[i] != '\n' and f[i+1] == '\n' and f[i+2] == '\n' and f[i+3] == '\n'):
                s1=f[i]
                s2=f[i+1]
                s3=f[i+2]
                s4=f[i+3]
                tu=s1+s2+s3+s4
                tu=tu.lstrip()
                o.append(tu)
            elif (f[i] != '\n' and f[i+1] == '\n' and f[i+2] == '\n'):
                s1=f[i]
                s2=f[i+1]
                s3=f[i+2]
                tu=s1+s2+s3
                tu=tu.lstrip()
                o.append(tu)
            elif (f[i] != '\n' and f[i+1] == '\n'):
                s1=f[i]
                s2=f[i+1]
                tu=s1+s2
                tu=tu.lstrip()
                o.append(tu)
            elif f[i] != '\n':
                f[i] = f[i].lstrip()
                o.append(f[i])
       
    ##remove all the empty strings
    while("" in o) :
        o.remove("")
  
    with open ('sample_dish.txt', 'w', encoding="utf-8") as f:
        for i in o:
            f.write(i)

    ##return the final list o
    return o

## Function2 - WorkFlow

#####    1>Takes the cleanned string 
#####     2>Count of the no of \n in each sentence  
#####     3>Append the count to each sentence
#####     4>Create a list of no of \n before each sentence
#####     5>creates dataframe with 3 columns  1:cleaned sstring without \n  2:no of \n's after sentence  3:no of \n before sentence 
#####     6>creates a csv file


In [3]:
def function2(cleanned_string):
    
    ## creating a list of no of \n after each sentence
    length=[]
    for i in range(len(cleanned_string)):
         length.append(len(re.findall('\n',cleanned_string[i])))
    
    ## creating list of of no of \n before each sentence
    l2=[0]
    for i in range(len(length)):
        l2.append(length[i])
    l2.pop(-1)

    for i in range(len(cleanned_string)):
            cleanned_string[i] = re.sub(r'\n'," ",cleanned_string[i])
            cleanned_string[i] = cleanned_string[i].strip()
        
    ## list of the sentence and the count 
    monk = []
    count= 0
    for data in cleanned_string:
        monk.append(  [data,length[count]]  )
        count = count + 1
        
    ##creating dataframe with 3 columns
    df = pd.DataFrame(monk)
    df['Sentence']=df[0]
    df['newline_after'] = df[1]
    df['newline_before'] = l2
    
    
    df['nwords'] = word_count(cleanned_string)
    df['numbers'] =digit_present(cleanned_string)
    df['nnumbers'] = digit_count(cleanned_string)
    df['nnouns'] = n_count(cleanned_string)
    df['nverbs'] = v_count(cleanned_string)
    df['nunits '] = unit_count(cleanned_string)
    
    df1 = df.drop([0,1], axis = 1)


    ## creating the csv file
    df1.to_csv('monk.csv',index = False)

In [13]:
### function call
function2(function1('https://www.indianhealthyrecipes.com/chicken-manchurian-recipe/'))

In [ ]:
1>https://www.yummytummyaarthi.com/paneer-manchurian-recipe-how-to-make/
2>https://www.vegrecipesofindia.com/paneer-manchurian-dry-recipe/
3>https://www.indianhealthyrecipes.com/paneer-manchurian-recipe/
4>https://hebbarskitchen.com/paneer-manchurian-dry-recipe/
5>https://foodviva.com/chinese-recipes/paneer-manchurian-recipe/
6>https://www.mydelicious-recipes.com/recipedetail/dry-paneer-manchurian
7>https://www.tarladalal.com/paneer-in-manchurian-sauce-86r
8>https://www.awesomecuisine.com/recipes/725/paneer-manchurian/

Create features like  ------- individual functions for each. input-> los output->list

1. Count of words
2. Presence of numbers    
3. Count of numbers
4. Count of nouns 
5. Count of units (create a dictionary of units like tsp, tbsp, etc.)

## Count of words

In [4]:
def word_count(list):
    word_count=[]
    for i in list:
        word_count.append(str(len(i.split())))
    return  word_count

## Presence of numbers

In [5]:
def hasNumber(stringVal):
    re_numbers = re.compile('\d')
    return 0 if (re_numbers.search(stringVal) == None) else 1
def digit_present(list):
    digit_present=[]
    for i in list:
        digit_present.append(hasNumber(i))
    return digit_present
    

## Count of numbers

In [6]:
def digit_count(list):
    digit_count=[]
    for i in list:
        digit_count.append(sum(c.isdigit() for c in i))
    return digit_count

## Count of nouns

In [7]:
def n_count(list):
    n_count=[]
    for i in list:
        n_count.append(sum(1 for word, pos in pos_tag(word_tokenize(i)) if pos.startswith("NN")))
    return  n_count

## Count of nouns

In [8]:
def v_count(list):
    v_count=[]
    for i in list:
        v_count.append(sum(1 for word, pos in pos_tag(word_tokenize(i)) if pos.startswith("VB")))
    return  v_count

## Count of units (create a dictionary of units like tsp, tbsp, etc.)

In [9]:
def unit_count(list):
    unit_count=[]
    measure_dict  = ["teaspoon","tablespoon","tablespoons","tblspn","inch","cups","cup","tbsps","tbsp","tsp","slices","slice","grams","litre","litres","medium","small"]
    for data in list:
        for sent in measure_dict :
            c =  0
            if sent in data:
                c = 1
                break
        unit_count.append(c)
    return unit_count
